# 2 Layered Neural Net – MNIST Dataset
## @psrth, 24.11.20

In [ ]:
# importing all relevant libraries
import numpy as np
import torch
import torchvision
import matplotlib.pyplot as plt
from time import time
from torchvision import datasets, transforms
from torch import nn, optim

# going to transform each image in the dataset
# converting to a RBG tensor, setting mean and deviation to 0.5
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

# loading the MNIST data set
trainset = datasets.MNIST('/content', download=True, train=True, transform=transform)
testset = datasets.MNIST('/content', download=True, train=False, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=True)


# building the neural network model

# the task dictates 2-Layer NN, so I've set the number of nodes in the layers:
# 784 -> 128 -> 64 -> 10
input_size = 784
hidden_sizes = [128, 64]
output_size = 10

model = nn.Sequential(nn.Linear(input_size, hidden_sizes[0]),
                      nn.ReLU(),
                      nn.Linear(hidden_sizes[0], hidden_sizes[1]),
                      nn.ReLU(),
                      nn.Linear(hidden_sizes[1], output_size),
                      nn.LogSoftmax(dim=1))

criterion = nn.NLLLoss()
images, labels = next(iter(trainloader))
images = images.view(images.shape[0], -1)

# log probabilities and loss
logps = model(images) 
loss = criterion(logps, labels) 

# pytorch optimizer
optimizer = optim.SGD(model.parameters(), lr=0.003, momentum=0.9)

# 10 epochs seemed fine for now
epochs = 10

for e in range(epochs):
    running_loss = 0
    for images, labels in trainloader:

        # flattening into a vector
        images = images.view(images.shape[0], -1)
    
        # the actual training pass
        optimizer.zero_grad()
        output = model(images)
        loss = criterion(output, labels)
        
        # backpropogating and weight adjustment
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()

correct_count, all_count = 0, 0

# time to test the model using PyTorch
for images,labels in testloader:
  for i in range(len(labels)):
    img = images[i].view(1, 784)
    with torch.no_grad():
        logps = model(img)

    ps = torch.exp(logps)
    probab = list(ps.numpy()[0])
    pred_label = probab.index(max(probab))
    true_label = labels.numpy()[i]
    if(true_label == pred_label):
      correct_count += 1
    all_count += 1

# printing result
print("Number Of Images Tested = ", all_count)
print("psrth's first neural network is ", (correct_count/all_count*100),"% accurate.")